In [58]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

找出适合OneHotEncoder的数据

https://zhuanlan.zhihu.com/p/686452650

In [59]:
# 加载数据
X_train = np.load('../Training data/X_train.npy')  # 特征 (1000, 111)
y_train = np.load('../Training data/y_train.npy')  # 目标 (1000, 11)


# 转换为Pandas DataFrame以方便处理
df = pd.DataFrame(X_train)
# 检查并填充NaN值
df.fillna(df.mean(), inplace=True)
# 特征缩放
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# 删除所有值都是NaN或0的列
# df_scaled.dropna(axis=1, how='all', inplace=True)
# df_scaled = df_scaled.loc[:, (df_scaled != 0).any(axis=0)]
# 不会有nan了 暂时也不想删掉0 看了一下数据感觉还是保留比较好


# 划分出来百分之二十的测试集
X_train, X_valid, Y_train, Y_valid = train_test_split(df_scaled, y_train, test_size=0.2, random_state=42)

# 将处理后的DataFrame转换回NumPy数组
X_train_processed = X_train.to_numpy()
# 现在X_train_processed是预处理后的数据，可以用于模型训练
len(X_train_processed[0])

111

Separate Model 部分

In [60]:
class SVMCustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=64, kernel="linear"):
        super(SVMCustomLayer, self).__init__()
        self.units = units
        self.kernel = kernel
        self.svm_model = np.array([SVC(kernel=self.kernel)]*11)

    def build(self, input_shape):
        # No trainable weights for SVM model

        # Fit SVM model to input data (assuming input_shape[-1] is the number of features)
        for i in range(11):
            self.svm_model[i].fit(tf.zeros((1, input_shape[-1])), [0])

    def call(self, inputs):
        # Use SVM model to predict
        y = [0]*11
        for i in range(11):
            y = self.svm_model[i].predict(inputs)
        return y

In [61]:
# Define the custom loss function
def custom_binary_crossentropy(y_true, y_pred):
    # Compute the binary cross entropy loss
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    # Take the mean across all labels
    return tf.reduce_mean(bce, axis=-1)

# Compile the model using the custom loss function
nnModel.compile(optimizer='adam', loss=custom_binary_crossentropy, metrics=['accuracy'])

NN部分

In [62]:
# 定义模型结构
nnModel = Sequential()
nnModel.add(Dense(128, input_dim=X_train_processed.shape[1], activation='relu'))
nnModel.add(Dense(64, activation='relu'))
nnModel.add(Dense(32, activation='relu'))
nnModel.add(SVMCustomLayer(32, kernel='linear'))
# 编译模型，添加精确率和召回率指标
nnModel.compile(
    optimizer='adam',
    loss='custom_binary_crossentropy', #将NN的值传入最终的separate models
    metrics=[
        tf.keras.metrics.Precision(name='precision', thresholds=[0.2, 0.4, 0.8]),
        tf.keras.metrics.Recall(name='recall', thresholds=[0.2, 0.4, 0.8]),
        tf.keras.metrics.Accuracy(name='accuracy')
    ]
)


a:\16_Environments\2_Python\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: The number of classes has to be greater than one; got 1 class

训练模型并输出

In [ ]:

# 训练模型
history = nnModel.fit(X_train, Y_train, epochs=100, validation_data=(X_valid, Y_valid))

# 计算模型在训练集上的精确率和召回率
train_accuracy = history.history['accuracy']
train_precision = history.history['precision'] 
train_recall = history.history['recall']

# 计算模型在验证集上的精确率和召回率
val_precision = history.history['val_precision']
val_recall = history.history['val_recall']
val_accuracy = history.history['val_accuracy']

# 打印训练集和验证集上的精确率和召回率
print(f'Train Accuracy: {train_accuracy[-1]}')
print(f'Train Precision: {train_precision[-1]}')
print(f'Train Recall: {train_recall[-1]}')
print(f'Validation Accuracy: {val_accuracy[-1]}')
print(f'Validation Precision: {val_precision[-1]}')
print(f'Validation Recall: {val_recall[-1]}')

Epoch 1/100


NotImplementedError: Cannot convert a symbolic tf.Tensor (sequential_6_1/dense_21_1/Sigmoid:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.